### https://towardsdatascience.com/how-to-forecast-sales-with-python-using-sarima-model-ba600992fa7d

In [1]:
import warnings
import itertools
import numpy as np
from numpy import concatenate, savetxt, unique, array, subtract
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
from pandas import merge, DataFrame, Series
import statsmodels.api as sm
import matplotlib
from math import sqrt
from statistics import mean
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'g'


In [2]:
df = pd.read_csv('CHL_Monthly_Stacked.csv', header=0, sep='[,]', parse_dates=True, squeeze=True, dayfirst=True, engine='python')
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
df.columns =['MeterNo', 'WeekNo', 'kWh']
y = df.set_index(['WeekNo'])


In [3]:
meter_ids = unique(y['MeterNo']) 
df2 = pd.DataFrame(index =['WeekNo'], columns=['prediction', 'actual', 'error','mse','rmse'])
pred_date = '2020-01-01'

In [4]:
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
print('Examples of parameter for SARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

Examples of parameter for SARIMA...
SARIMAX: (0, 0, 1) x (0, 0, 1, 12)
SARIMAX: (0, 0, 1) x (0, 1, 0, 12)
SARIMAX: (0, 1, 0) x (0, 1, 1, 12)
SARIMAX: (0, 1, 0) x (1, 0, 0, 12)


In [6]:
for m in meter_ids:
    df = pd.read_csv('CHL_Monthly_Stacked.csv', header=0, sep='[,]', parse_dates=True, squeeze=True, dayfirst=True, engine='python')
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
    df.columns =['MeterNo', 'WeekNo', 'kWh']
    y = df.set_index(['WeekNo'])
    y = y.loc[y['MeterNo'] == m]
    y.drop(y.columns[[0]], axis = 1, inplace = True)
    mod = sm.tsa.statespace.SARIMAX(y,
                                order=(1, 1, 1),
                                seasonal_order=(0, 1, 1, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
    results = mod.fit()
    pred = results.get_prediction(start=pd.to_datetime(pred_date), dynamic=False)
    y_forecasted = pred.predicted_mean
    y_forecasted =y_forecasted.to_frame()
    y_truth = y[pred_date:]
    df1 = merge(y_forecasted, y_truth, left_index=True, right_index=True)
    df1.columns =['prediction', 'actual']
    df1['error'] = round(((df1['prediction'] - df1['actual'])),2)
    df1['mse'] = round(df1['error'] ** 2,2)
    df1['rmse'] = round(np.sqrt(df1['mse']),2)
    print(df1)
    df2 = df2.append(df1)

C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'


            prediction  actual  error     mse   rmse
2020-01-01  191.275406     181  10.28  105.68  10.28
2020-02-01  155.619192     157  -1.38    1.90   1.38
2020-03-01  147.879303     121  26.88  722.53  26.88
2020-04-01   80.473386      80   0.47    0.22   0.47
2020-05-01   58.924402      72 -13.08  171.09  13.08
2020-06-01   59.676918      61  -1.32    1.74   1.32
2020-07-01   57.653148      59  -1.35    1.82   1.35
2020-08-01   56.532348      55   1.53    2.34   1.53
2020-09-01   57.288286      55   2.29    5.24   2.29
2020-10-01   60.936712      61  -0.06    0.00   0.00
2020-11-01  113.270564     117  -3.73   13.91   3.73
2020-12-01  167.699991     192 -24.30  590.49  24.30
2021-01-01  198.569989     222 -23.43  548.96  23.43
2021-02-01  214.531268     239 -24.47  598.78  24.47


C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'


             prediction  actual   error       mse    rmse
2020-01-01   613.461930     635  -21.54    463.97   21.54
2020-02-01   609.243575     644  -34.76   1208.26   34.76
2020-03-01   552.078023     477   75.08   5637.01   75.08
2020-04-01   319.074996     294   25.07    628.50   25.07
2020-05-01   245.555460     291  -45.44   2064.79   45.44
2020-06-01   269.419266     198   71.42   5100.82   71.42
2020-07-01   140.279584     169  -28.72    824.84   28.72
2020-08-01   174.772038     182   -7.23     52.27    7.23
2020-09-01   219.932497     227   -7.07     49.98    7.07
2020-10-01   274.625357     359  -84.37   7118.30   84.37
2020-11-01   549.467905     616  -66.53   4426.24   66.53
2020-12-01   779.608439     953 -173.39  30064.09  173.39
2021-01-01  1070.095274     965  105.10  11046.01  105.10
2021-02-01   868.202087     886  -17.80    316.84   17.80


C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'


...

             prediction  actual   error       mse    rmse
2020-01-01   646.686986     620   26.69    712.36   26.69
2020-02-01   574.621635     591  -16.38    268.30   16.38
2020-03-01   481.532167     449   32.53   1058.20   32.53
2020-04-01   287.534903     290   -2.47      6.10    2.47
2020-05-01   193.644130     253  -59.36   3523.61   59.36
2020-06-01   193.944237     214  -20.06    402.40   20.06
2020-07-01   203.899535     203    0.90      0.81    0.90
2020-08-01   188.075709     190   -1.92      3.69    1.92
2020-09-01   215.362239     299  -83.64   6995.65   83.64
2020-10-01   399.524861     591 -191.48  36664.59  191.48
2020-11-01   826.847748     803   23.85    568.82   23.85
2020-12-01   949.955310    1101 -151.04  22813.08  151.04
2021-01-01  1197.859818    1081  116.86  13656.26  116.86
2021-02-01   997.329647     949   48.33   2335.79   48.33


C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'


            prediction  actual   error       mse    rmse
2020-01-01  531.993815     491   40.99   1680.18   40.99
2020-02-01  366.194314     405  -38.81   1506.22   38.81
2020-03-01  354.849794     356   -1.15      1.32    1.15
2020-04-01  234.393387     283  -48.61   2362.93   48.61
2020-05-01  164.984270     219  -54.02   2918.16   54.02
2020-06-01  167.214381     161    6.21     38.56    6.21
2020-07-01  144.893249     156  -11.11    123.43   11.11
2020-08-01  160.802531     148   12.80    163.84   12.80
2020-09-01  163.560088     183  -19.44    377.91   19.44
2020-10-01  225.821035     281  -55.18   3044.83   55.18
2020-11-01  594.994764     377  217.99  47519.64  217.99
2020-12-01  438.162655     515  -76.84   5904.39   76.84
2021-01-01  520.867699     559  -38.13   1453.90   38.13
2021-02-01  483.894279     561  -77.11   5945.95   77.11


C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\Paul\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'


             prediction  actual   error       mse    rmse
2020-01-01  1044.177772    1223 -178.82  31976.59  178.82
2020-02-01  1188.845938    1323 -134.15  17996.22  134.15
2020-03-01  1166.914063     969  197.91  39168.37  197.91
2020-04-01   651.970726     583   68.97   4756.86   68.97
2020-05-01   400.636344     437  -36.36   1322.05   36.36
2020-06-01   325.044581     306   19.04    362.52   19.04
2020-07-01   302.222459     286   16.22    263.09   16.22
2020-08-01   280.875716     264   16.88    284.93   16.88
2020-09-01   325.007804     368  -42.99   1848.14   42.99
2020-10-01   450.761281     653 -202.24  40901.02  202.24
2020-11-01  1055.065376    1114  -58.93   3472.74   58.93
2020-12-01  1418.731250    1721 -302.27  91367.15  302.27
2021-01-01  2024.282871    1797  227.28  51656.20  227.28
2021-02-01  1738.689957    1719   19.69    387.70   19.69


In [7]:
# Each meter per week
df2 = df2.dropna()
print(df2)

             prediction actual   error       mse    rmse
2020-01-01   191.275406    181   10.28    105.68   10.28
2020-02-01   155.619192    157   -1.38      1.90    1.38
2020-03-01   147.879303    121   26.88    722.53   26.88
2020-04-01    80.473386     80    0.47      0.22    0.47
2020-05-01    58.924402     72  -13.08    171.09   13.08
...                 ...    ...     ...       ...     ...
2020-10-01   450.761281    653 -202.24  40901.02  202.24
2020-11-01  1055.065376   1114  -58.93   3472.74   58.93
2020-12-01  1418.731250   1721 -302.27  91367.15  302.27
2021-01-01  2024.282871   1797  227.28  51656.20  227.28
2021-02-01  1738.689957   1719   19.69    387.70   19.69

[3990 rows x 5 columns]


In [8]:
# Summary of each meter for each week
def Summary(x):
    return round(Series(index=['min','max', 'mean','sum','count'],data=[x.min(),x.max(),x.mean(),x.sum(),x.count()]),2)
df2.apply(Summary)

,prediction,actual,error,mse,rmse
min,-1977.60,-1821.00,-1255.60,0.00,0.00
max,2230.65,2152.00,1249.00,1576531.36,1255.60
mean,397.61,395.77,1.84,6682.50,49.48
sum,1586457.25,1579125.00,7332.20,26663171.49,197435.89
count,3990.00,3990.00,3990.00,3990.00,3990.00


In [9]:
df2.drop(df2.columns[[2,3,4]], axis = 1, inplace = True)
print(df2)

             prediction actual
2020-01-01   191.275406    181
2020-02-01   155.619192    157
2020-03-01   147.879303    121
2020-04-01    80.473386     80
2020-05-01    58.924402     72
...                 ...    ...
2020-10-01   450.761281    653
2020-11-01  1055.065376   1114
2020-12-01  1418.731250   1721
2021-01-01  2024.282871   1797
2021-02-01  1738.689957   1719

[3990 rows x 2 columns]


In [10]:
df2a = df2.groupby(df2.index)['prediction'].sum()
df2b = df2.groupby(df2.index)['actual'].sum()
df3 = merge(df2a, df2b, left_index=True, right_index=True)
print(df3)

               prediction  actual
2020-01-01  161625.747270  157644
2020-02-01  145662.201194  153167
2020-03-01  144876.256473  132075
2020-04-01  104765.692205  102761
2020-05-01   80535.176466   86880
2020-06-01   71011.878136   70208
2020-07-01   65346.233758   65112
2020-08-01   61382.345226   59640
2020-09-01   72992.078193   68140
2020-10-01   80588.959068   97677
2020-11-01  136550.603036  121306
2020-12-01  143299.414317  157891
2021-01-01  173304.817659  159650
2021-02-01  144515.850499  146974


In [11]:
# Sum of all meters for each week
df3['error'] = round(((df3['prediction'] - df3['actual'])),2)
df3['mse'] = df3['error'] ** 2
df3['rmse'] = round(np.sqrt(df3['mse']),2)
df3['%error'] = (df3['error']/df3['actual'])*100
print(df3)
df3.to_csv('CHL_SARIMA_AllMeters_Monthly.csv')

               prediction  actual     error           mse      rmse     %error
2020-01-01  161625.747270  157644   3981.75  1.585433e+07   3981.75   2.525786
2020-02-01  145662.201194  153167  -7504.80  5.632202e+07   7504.80  -4.899750
2020-03-01  144876.256473  132075  12801.26  1.638723e+08  12801.26   9.692417
2020-04-01  104765.692205  102761   2004.69  4.018782e+06   2004.69   1.950828
2020-05-01   80535.176466   86880  -6344.82  4.025674e+07   6344.82  -7.302970
2020-06-01   71011.878136   70208    803.88  6.462231e+05    803.88   1.144998
2020-07-01   65346.233758   65112    234.23  5.486369e+04    234.23   0.359734
2020-08-01   61382.345226   59640   1742.35  3.035784e+06   1742.35   2.921445
2020-09-01   72992.078193   68140   4852.08  2.354268e+07   4852.08   7.120751
2020-10-01   80588.959068   97677 -17088.04  2.920011e+08  17088.04 -17.494436
2020-11-01  136550.603036  121306  15244.60  2.323978e+08  15244.60  12.567062
2020-12-01  143299.414317  157891 -14591.59  2.12914

In [12]:
# Summary of the sum of all meters for each week
df3.apply(Summary)

,prediction,actual,error,mse,rmse,%error
min,61382.35,59640.00,-17088.04,5.486369e+04,234.23,-17.49
max,173304.82,159650.00,15244.60,2.920011e+08,17088.04,12.57
mean,113318.38,112794.64,523.73,8.838670e+07,7379.08,0.44
sum,1586457.25,1579125.00,7332.26,1.237414e+09,103307.06,6.22
count,14.00,14.00,14.00,1.400000e+01,14.00,14.00


for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(y,order=param,seasonal_order=param_seasonal,enforce_stationarity=False,enforce_invertibility=False, period=1)
            results = mod.fit()
            print('ARIMA{}x{}12 - AIC:{}'.format(param,param_seasonal,results.aic))
        except: 
            continue

for m in meter_ids:
    df = pd.read_csv('CHL_Monthly_Stacked.csv', header=0, sep='[,]', parse_dates=True, squeeze=True, dayfirst=True, engine='python')
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
    df.columns =['MeterNo', 'WeekNo', 'kWh']
    y = df.set_index(['WeekNo'])
    y = y.loc[y['MeterNo'] == m]
    y.drop(y.columns[[0]], axis = 1, inplace = True)
    #print(y)
    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                mod = sm.tsa.statespace.SARIMAX(y,order=param,seasonal_order=param_seasonal,enforce_stationarity=False,enforce_invertibility=False)
                results = mod.fit()
                print('ARIMA{}x{}12 - AIC:{}'.format(param,param_seasonal,results.aic))
            except: 
                continue